In [9]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')


In [10]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [6]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [30]:
Geocoder_package=pd.read_csv('https://cocl.us/Geospatial_data')
Geocoder_package.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
Geocoder_package=Geocoder_package.rename(index=str, columns={"Postal Code":"PostalCode"})
Geocoder_package.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [78]:
toronto_dict = {'PostalCode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}

In [79]:
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
1,M5M,North York,Bedford Park
2,M6M,York,"Keelsdale, Mount Dennis, Silverthorn"
3,M5S,Downtown Toronto,University of Toronto
4,M6P,West Toronto,High Park


In [92]:
result=pd.merge(Geocoder_package,df_toronto,on='PostalCode')
result=result[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
result.to_csv('toronto_base.csv')
result.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 2 of assignment

In [93]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes 

In [97]:

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import folium # map rendering library
print('Libraries imported.')


Libraries imported.


In [98]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [110]:
CLIENT_ID = '1RCC5Q1JCPIEJ5O1BGHVKPABSLXWETBNEJUUK54MH4CNGM0Y' # your Foursquare ID
CLIENT_SECRET = 'SD44P3NM3WV1H0SZOHE04M2ZYEB1YK2IZTJMYF0HULE3CCCD' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1RCC5Q1JCPIEJ5O1BGHVKPABSLXWETBNEJUUK54MH4CNGM0Y
CLIENT_SECRET:SD44P3NM3WV1H0SZOHE04M2ZYEB1YK2IZTJMYF0HULE3CCCD


## Create a new data frame with neighborhoods in Scarbor

In [104]:
Scarboroughs = result[result['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarboroughs.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Create a map of Scarborough

In [105]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [109]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Scarboroughs['Latitude'], Scarboroughs['Longitude'], Scarboroughs['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

## Get the top 100 venues in the neighborhood from Scarborough

In [112]:
neighborhood_latitude = Scarboroughs.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = Scarboroughs.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = Scarboroughs.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.806686299999996, -79.19435340000001.


In [114]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=1RCC5Q1JCPIEJ5O1BGHVKPABSLXWETBNEJUUK54MH4CNGM0Y&client_secret=SD44P3NM3WV1H0SZOHE04M2ZYEB1YK2IZTJMYF0HULE3CCCD&ll=43.773077,-79.257774&v=20180604&radius=500&limit=100'

In [115]:
outcome = requests.get(url).json()
outcome

{'meta': {'code': 200, 'requestId': '5bdd78009fb6b71cc5eb76c6'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 39,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [116]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [117]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = outcome['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,DAVIDsTEA,Tea Room,43.776613,-79.258516
2,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
6,Hot Topic,Clothing Store,43.775450,-79.257929
7,Scarborough Town Centre,Shopping Mall,43.775231,-79.257462
8,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
9,American Eagle Outfitters,Clothing Store,43.775908,-79.258352


In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
scarborough_venues = getNearbyVenues(names=Scarboroughs['Neighbourhood'],
                                   latitudes=Scarboroughs['Latitude'],
                                   longitudes=Scarboroughs['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside
Birch Cliff
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Tam O'Shanter
Agincourt North, Milliken
Steeles West
Upper Rouge


In [126]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [127]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [128]:
scarb_onehot.shape


(87, 57)

In [129]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)


,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,Agincourt,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0,0.00,0.000000,0.000000,0.25,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.000000,0.0
1,"Agincourt North, Milliken",0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,0.000000,0.0,0.0,0.5,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
2,Birch Cliff,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.25,0.000000,0.000000,0.00,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.000000,0.0
3,Cedarbrae,0.000000,0.142857,0.0,0.142857,0.142857,0.0,0.00,0.0,0.0,0.00,0.142857,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.00,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.142857,0.0,0.000000,0.0
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000000,0.0,0.200000,0.000000,0.0,0.00,0.2,0.1,0.00,0.000000,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.1,0.000000,0.0,0.100000,0.0,0.0,0.0,0.000000,0.00,0.0,0.1,0.0,0.0,0.000000,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0.000000,0.0,0.000000,0.0
5,"Cliffcrest, Cliffside",0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0
6,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.166667,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0,0.166667,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.166667,0.0


In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,College Stadium,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, Milliken",Playground,Park,Women's Store,Coffee Shop,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Women's Store,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Construction & Landscaping,Gym,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Ice Cream Shop,Soccer Field,Park,Intersection,Metro Station,Bus Station,Discount Store,Convenience Store
5,"Cliffcrest, Cliffside",American Restaurant,Motel,Intersection,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Pet Store,Chinese Restaurant,Women's Store,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint
7,"Highland Creek, Rouge Hill, Port Union",Bar,Women's Store,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,"Ionview, Kennedy Park",Discount Store,Train Station,Bus Station,Department Store,Convenience Store,Hobby Shop,Coffee Shop,Women's Store,College Stadium,Gym
9,"Maryvale, Wexford",Middle Eastern Restaurant,Vietnamese Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Breakfast Spot,Electronics Store,Construction & Landscaping,Convenience Store,Department Store


## Run k-means to cluster the neighborhoods

In [132]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 3, 1, 1, 1, 1, 1, 2, 1, 1], dtype=int32)

In [133]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,Women's Store,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711,1,Mexican Restaurant,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Women's Store,Grocery Store,General Entertainment,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Insurance Office,Korean Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Construction & Landscaping,Gym,Grocery Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Women's Store,Playground,Construction & Landscaping,Coffee Shop,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029,1,Discount Store,Train Station,Bus Station,Department Store,Convenience Store,Hobby Shop,Coffee Shop,Women's Store,College Stadium,Gym
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Park,Intersection,Metro Station,Bus Station,Discount Store,Convenience Store
8,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476,1,American Restaurant,Motel,Intersection,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,M1N,Scarborough,Birch Cliff,43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Women's Store,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant


## Visualization

In [134]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Example of each K

In [137]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,0,Middle Eastern Restaurant,Vietnamese Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Breakfast Spot,Electronics Store,Construction & Landscaping,Convenience Store,Department Store


In [138]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Women's Store,Vietnamese Restaurant,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store
2,Scarborough,1,Mexican Restaurant,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Women's Store,Grocery Store,General Entertainment,Fried Chicken Joint
3,Scarborough,1,Coffee Shop,Insurance Office,Korean Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,1,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Construction & Landscaping,Gym,Grocery Store
5,Scarborough,1,Women's Store,Playground,Construction & Landscaping,Coffee Shop,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,Scarborough,1,Discount Store,Train Station,Bus Station,Department Store,Convenience Store,Hobby Shop,Coffee Shop,Women's Store,College Stadium,Gym
8,Scarborough,1,American Restaurant,Motel,Intersection,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Women's Store,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Pet Store,Chinese Restaurant,Women's Store,College Stadium,Grocery Store,General Entertainment,Fried Chicken Joint
13,Scarborough,1,Pizza Place,Chinese Restaurant,Pharmacy,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Noodle House,Italian Restaurant,College Stadium,Construction & Landscaping


In [139]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,2,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Park,Intersection,Metro Station,Bus Station,Discount Store,Convenience Store


In [140]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,Women's Store,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,Scarborough,3,Clothing Store,Skating Rink,Breakfast Spot,Lounge,College Stadium,Gym,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


In [141]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Scarborough,4,Chinese Restaurant,Fast Food Restaurant,Breakfast Spot,Grocery Store,Japanese Restaurant,Coffee Shop,Pharmacy,Pizza Place,American Restaurant,Gym
